In [ ]:
# 3_qlora_finetuning.ipynb
"""
Fine-tune LLaMA 3 (8B) using QLoRA on a small Indigenous language dataset.
Adapted for low-resource training using Hugging Face + bitsandbytes.
"""

In [ ]:

# 📦 Step 1: Install dependencies (run only if needed)
!pip install transformers datasets peft accelerate bitsandbytes trl

In [ ]:
# 🧠 Step 2: Import libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
import torch

In [ ]:
# 🧾 Step 3: Load dataset
dataset = load_dataset("text", data_files="../datasets/sample_corpus.txt", split="train")

# Optional preprocessing
def format_example(example):
    return {"text": example["text"] + " </s>"}

dataset = dataset.map(format_example)

In [ ]:
# 📚 Step 4: Load tokenizer and model
model_name = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

In [ ]:
# 🛠️ Step 5: Prepare model for QLoRA fine-tuning
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# 🧪 Step 6: Define training arguments
training_args = TrainingArguments(
    output_dir="../models/llama3-8b-qlora-output",
